# CPSC 589 Artificial Neural Networks Project 2
## Handwritten Character Recognition

### Group Members: 
### Jeremy Rico; jjrico@csu.fullerton.edu
### Daniel Walsh; danielwalsh27@csu.fullerton.edu
### Haojie Pan; haojie@csu.fullerton.edu

This program uses as an Artifical Neural Network with one hidden layer to predict the identity of handwritten characters of all 26 letters of the english alphabet.

Dataset: EMNIST Letters - contains 145,600 28x28px images of handwritten characters divided into 26 classes. The dataset is divided into 108,000 training samples, 20,800 validation samples, and 20,800 test samples. More information about the dataset can be found here: https://www.nist.gov/itl/products-and-services/emnist-dataset

Our project utilizes the tf.keras module of TensorFlow to create, train, and test the neural network.

In [19]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from os.path import dirname, join as pjoin
from scipy import io as sio
#from keras.utils.np_utils import to_categorical

In [20]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import datasets
from tensorflow.keras import optimizers

## Data Manipulation

In this section we access the emnist data set using sio.loadmat(). The 28x28px image has been flattened into a 784 vector.

We then divide the dataset into training, validation, and test sets. The emnist data is already divided into training and test sets. However, we obtain the validation set from taking the 5% off the end of the training data (so that it is the same size as the test set). 

Finally, we convert each of the label vectors to categorical arrays that can be recognized by the keras model we will be creating. Therefore, for each 28x28px image (or 784 1D vector) there is exactly one vector of length 26 to label which letter of the alphabet is represented.

In [21]:
# Define some vars we will use to construct the model
num_classes = 26
batch_size = 2000
epochs = 20
neurons = 1000

mat_contents = sio.loadmat('ANN-Handwritten-Char-Recognition-master/matlab/emnist-letters.mat')
data = mat_contents['dataset']

X_train = data['train'][0,0]['images'][0,0]
y_train = data['train'][0,0]['labels'][0,0]
X_test = data['test'][0,0]['images'][0,0]
y_test = data['test'][0,0]['labels'][0,0]

val_start = X_train.shape[0] - X_test.shape[0]
X_val = X_train[val_start:X_train.shape[0],:]
y_val = y_train[val_start:X_train.shape[0]]
X_train = X_train[0:val_start,:]
y_train = y_train[0:val_start]

y_train -=1
y_val -=1
y_test -=1

#convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(X_train.shape[0], "training samples")
print(X_val.shape[0], "validation samples")
print(X_test.shape[0], "test samples")
print(X_val.shape, y_val.shape)

104000 training samples
20800 validation samples
20800 test samples
(20800, 784) (20800, 26)


## Model Initialization

Here, we create the dense neural network layer by layer, using tr.keras.

The model is first initialized with a 784 neuron inpupt layer. 

The amount of neurons in the hidden layer was somethings that we played around with a lot. See the table below for all hyperparameters and how they affected accuracy. We eventually settled on a hidden layer with 1000 neurons to maximize efficiency and accuracy.

The final layer consists of 26 neurons (one for each letter) and softmax activation.

In [22]:
model = tf.keras.Sequential()
model.add(layers.Dense(neurons, activation='relu', input_shape=(784,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(neurons, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 1000)              785000    
_________________________________________________________________
dropout_8 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
dropout_9 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 26)                26026     
Total params: 1,812,026
Trainable params: 1,812,026
Non-trainable params: 0
_________________________________________________________________


## Model Compilation and Training

Here the model is compiled and trained with the parameters that we finalized.

This table shows some insight into the different hyperparameters we tested, and the impact they had on model accuracy:

| Epochs | Batch Size | Hidden Layer Neurons | Accuracy |
|--------|------------|----------------------|----------|
|20      |1000        |800                   |89.69%    |
|20      |2000        |800                   |89.56%    |
|20      |1000        |500                   |88.58%    |
|20      |1000        |1000                  |89.85%    |
|20      |2000        |1000                  |90.25%    |
|20      |2000        |800                   |89.74%    |
|20      |2000        |1200                  |90.27%    |
|20      |2500        |1200                  |89.64%    |
|15      |2000        |1200                  |89.67%    |
|20      |1500        |1200                  |89.56%    |
|20      |1500        |800                   |89.57%    |
|20      |1000        |800                   |89.18%    |
|20      |2000        |800                   |89.81%    |
|20      |2000        |1200                  |89.62%    |
|20      |2000        |1024                  |89.23%    |
|20      |2000        |1000                  |90.22%    |

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (X_val, y_val))

Train on 104000 samples, validate on 20800 samples
Epoch 1/20
104000/104000 [==============================] - 6s 54us/sample - loss: 29.4035 - accuracy: 0.4767 - val_loss: 1.5905 - val_accuracy: 0.6753
Epoch 2/20
104000/104000 [==============================] - 6s 54us/sample - loss: 1.7995 - accuracy: 0.6292 - val_loss: 1.0604 - val_accuracy: 0.7439
Epoch 3/20
104000/104000 [==============================] - 6s 54us/sample - loss: 1.2654 - accuracy: 0.7088 - val_loss: 0.8173 - val_accuracy: 0.7878
Epoch 4/20
104000/104000 [==============================] - 6s 56us/sample - loss: 0.8763 - accuracy: 0.7690 - val_loss: 0.6364 - val_accuracy: 0.8112
Epoch 5/20
104000/104000 [==============================] - 6s 55us/sample - loss: 0.6738 - accuracy: 0.8095 - val_loss: 0.5401 - val_accuracy: 0.8438
Epoch 6/20
104000/104000 [==============================] - 6s 56us/sample - loss: 0.5571 - accuracy: 0.8370 - val_loss: 0.4975 - val_accuracy: 0.8599
Epoch 7/20
104000/104000 [================

### Model Evaluation

The model is evaluated by using the test set and the tf.keras() module. We output the final loss and model accuracy.

In [24]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.4330102442536568
Test accuracy: 0.9022596


### Review

After trying many different hyperparameter values we eventually settled on the ones that gave us the highest prediction accuracy. We came to our final values by testing out a different number of epochs, batch size, and hidden neurons to reach an accuracy of 90.22%. Our final values were as follows:

Epochs: 20

Batch Size: 2000

Hidden Neurons: 1000

Some of the challenges that arose during this project were the manipulation of the dataset and creating the model so that it would run properly. For instance, a problem arose where we couldn't get the model to work with 26 classes because we had divided the training and validation data wrong. It was because of this that the model wouldn't run with the proper number of classes. We had to do some research into the EMNIST data set and how it was formatted to finally get the model working properly.
